In [ ]:
import osiris_utils as ou
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
plt.rcParams['font.size'] = 14

from mpl_toolkits.mplot3d import Axes3D

import contextlib
import io


In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)

    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.3f}$".format(t[0]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
        label2 = ("$t = {:.3f}$".format(t[-1]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))


    if label == "Baseline":
        color = "black"
        markersize = 0
        linewidth = 0.6
    else:
        color = None
        markersize = 1.5
        linewidth = 0.8
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle='-', markersize=markersize, linewidth=linewidth, label=label, color=color)
    
    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x1"]))
    ax.set_ylabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x2"]))
    ax.set_zlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x3"]))
    ax.legend()

    return fig, ax

In [ ]:
# raw = ou.OsirisRawFile("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/MS/RAW/electrons/RAW-electrons-000003.h5")
# raw.raw_to_file_tags("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/file_tags.tags", type="random", n_tags=5)

## Gyromotion

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/gyro_Gca/pressure_2D.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/gyro_Boris/pressure_2D.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label="boris")
ax.plot(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label = "boris")
ax.scatter(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"Gyromotion")


plt.show()

## EXB

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/EXB_Gca/pressure_2D.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/EXB_Boris/pressure_2D.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label="boris")
ax.plot(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label = "boris")
ax.scatter(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label = "gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["x1"]))
ax.set_ylabel("${}$".format(boris["electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["x2"]))


plt.show()

fig, ax = plt.subplots()
ax.plot(boris["electrons"]["tracks"]["t"][particle,:], boris["electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["electrons"]["tracks"]["t"][particle,:], gca["electrons"]["tracks"]["x1"][particle,:], label="gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

## EXB 3d

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EXB_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EXB_Boris/Boris.in")

In [ ]:
fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=10000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=10000, fig=fig, ax=ax)

In [ ]:
particle = 3

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x1"][particle,:], boris["test_electrons"]["tracks"]["x2"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x1"][particle,:], gca["test_electrons"]["tracks"]["x2"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x1"][particle,:], boris["test_electrons"]["tracks"]["x2"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x1"][particle,:], gca["test_electrons"]["tracks"]["x2"][particle,:], label = "gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x2"]))

plt.show()


fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x1"][particle,:], label="gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

## Gyromotion 3d

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Gyro_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Gyro_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"Gyromotion")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=10, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=10, fig=fig, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x1"][particle,:], label="gca")

ax.set_title(r"Gyromotion")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## Curv 3d

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Curv_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Curv_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"Curvature")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=10, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=10, fig=fig, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"Curvature")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## $\nabla B$ Drift

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_GradB_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_GradB_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"$\nabla B$ Drift")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x1"][particle,:], label="gca")

ax.set_title(r"$\nabla B$ Drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## ( V_E . Grad ) b and parallel terms

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"( E . Grad ) b")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"( E . Grad ) b")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v2_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v2_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"( E . Grad ) b")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"( E . Grad ) b")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v3_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v3_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"( E . Grad ) b")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"( E . Grad ) b")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()